学生証番号 :

名前 :

メールアドレス(法政大学) :

# B2-5 VGG16ネットワークを用いた学習(2) 転移学習、Fine-tuning

拡大したCIFAR10画像をVGGネットワークの構造で学習する。

B2-4の実験では、たった数回の学習でもとても時間がかかった。
これは深層学習モデルのパラメータがとても多く、計算量が大きいからである。

今回は、事前学習により得られた重み（パラメータ）を効果的に活用する転移学習と呼ばれる、実践的な学習法を体験する。

転移学習では、すでに得られている重み（今回だと画像中の特徴を捉える画像フィルタ）をそのまま活用し、一部分のみ再学習する。


具体的には、本当の目的（CIFAR10)とは異なるImageNetで学習された重みを活用し、最後の12800$\times$10の部分のみを学習する。



<img src="https://drive.google.com/uc?export=view&id=1G0WQ5MaVpHRCktqHBiGbHARPb_MlGIDd" width = 50%></img>

In [ ]:
# [1-0]
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg16

## データセットの準備

In [ ]:
# [1-1]
def get_data(data_name="CIFAR10"):
    if data_name == "CIFAR100":
        (X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
    else:
        (X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
    y_train = y_train.flatten()
    y_test = y_test.flatten()
    return (X_train, y_train), (X_test, y_test)

In [ ]:
# [1-2]
# 予測ラベルの名前を所持しておく
target_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]


def random_plot(X, y, predict=None):
    W = 10
    H = 5
    fig = plt.figure(figsize=(10, 20))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=0.3, hspace=0.10, wspace=0.10)
    for i in range(5):
        for j in range(10):
            x_tmp = X[y == j]
            idx = np.random.randint(len(x_tmp))
            x = x_tmp[idx].reshape(32, 32, 3)
            ax = fig.add_subplot(H, W, (i * 10) + j + 1, xticks=[], yticks=[])
            ax.imshow(x)
            if predict is not None:
                pred_tmp = predict[y == j]
                p = pred_tmp[idx]
                ax.set_title(f"{j} -> {p}")
            else:
                ax.set_title(target_names[j])

In [ ]:
# [1-3]
DATASET = "CIFAR10"
# CIFAR100を使用したい場合は以下をコメントアウト
# CIFAR100の場合はrandom_plotが全てのラベルを表示しないので注意
# DATASET = "CIFAR100"
(X_train, y_train), (X_test, y_test) = get_data(DATASET)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# [1-4]
# 学習用データを覗いてみる。
# 複数回実行すると画像も変化する。
random_plot(X_train, y_train)

## 学習済みのVGG16ネットワークのダウンロード

B02-2と同様に、ImageNetで学習済みのネットワークをダウンロードする。
include_top=Falseとして、全結合層の部分を取り外している。  
つまり、Conv5_3をmax poolしたところまでのネットワークになっている。

B02-3と、まったく同じ構造だが、前回との違いは、今回のモデルが**学習済み**であることである。




In [ ]:
# [1-5]
# VGG16をCIFAR10ように改良したモデルを読み込む関数を定義
def get_vgg16_model(data_name="CIFAR10"):
    num_output = 10 if data_name == "CIFAR10" else 100
    # change input image size 224x224 -> 160x160
    # conv5_3のあとのmax poolまでのネットワーク(core)の出力をflattenした後、
    # 全結合層を介して10次元の出力をつなげる
    model = keras.Sequential(
        [
            keras.Input(shape=(None, None, 3)),
            keras.layers.Lambda(
                lambda img: tf.image.resize(img, (160, 160)), name="resize"
            ),
            keras.layers.Lambda(vgg16.preprocess_input, name="preprocess"),
            # VGGのCNN部分
            vgg16.VGG16(include_top=False, weights="imagenet"),
            keras.layers.Flatten(),
            keras.layers.Dense(num_output, activation="softmax"),
        ]
    )
    return model

In [ ]:
# [1-6]
# モデルの読みこみ
model = get_vgg16_model(DATASET)

# モデルの全体図の概要
model.summary()

In [ ]:
# [1-7]
# VGG部分の概要確認
model.layers[2].summary()

## モデルセットアップと重みの固定

今までのmodel_setup関数を改良し、学習する層を変更できるようにしてある。
今回は全結合層(dense)部分のみ学習を行うため、training引数には"dense"を指定する。
学習される重みの数を確認し、B2-4で行ったゼロからの学習（scratch学習）の時と比較せよ。

学習の対象重みの数： (12800 + bias 1) x 10 (output)

In [ ]:
# [1-8]
# 誤差関数と最適化手法を設定する関数を作成する。
# 新しく、学習する層を指定するtrainingという引数を用意。
def model_setup(model, training="all", optimizer="adam", lr=0.001):
    if optimizer == "adam":
        optim = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = keras.optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError
    # 学習する層の選択
    if training == "dense":
        model.layers[2].trainable = False
    elif training == "deep":
        for i in range(15):
            model.layers[2].layers[i].trainable = False
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"]
    )

In [ ]:
# [1-9]
# denseの部分だけ学習するときはtrainingに"dense"を指定する。
model_setup(model, training="dense")
# 学習されるパラメータの数を確認しておく。
# （これでも大規模な全結合層　flatten-4096-4096-1000 を外しているので総数は普通のVGG16 の1/10程度になっている）
model.summary()

# VGG16の事前学習の重み使用した転移学習
最初に学習前のモデルで予測を行い、性能を確認する。

In [ ]:
# [1-10]
# テストデータで評価する
loss_no_train, accuracy_no_train = model.evaluate(X_test, y_test, verbose=0)
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

予測確率を出すdense部分が学習できていないため、ランダムな予測と同じ結果になる。

## モデルの学習
B2-3 の全体を学習するスクラッチ学習と比べ学習するパラメータが少ない分、学習時間を短縮できるが、スクラッチと同様に学習には時間がかかるため、実験時間内では3 epochのみの学習しか行わない。 学習の傾向に関しては事前に実行した時のログから確認する。

※ レポートには10epochで学習したモデルの結果のグラフや、予測などを確認してほしいので授業外でEPOCHの値を10にして全体を実行しなおすこと。すべて実行完了するまでに１時間ほどかかるが、ヘッダーの「ランタイム」タブの「再起動してすべてのセルを実行」を行うことで全体を実行してくれる。
１時間放置していると自動でセッションが切れてしまうので、定期的に確認すること。

In [ ]:
# [1-11]
EPOCH = 3
# ここを10にして再度実行する。
# EPOCH = 10
batch_size = 128

trainlog = model.fit(
    X_train, y_train, epochs=EPOCH, validation_split=0.1, batch_size=batch_size
)

以下が10EPOCH回した時のログになっている。
```
```
転移学習を使用することで1EPOCHめから高い性能で予測できていることが確認できた。
また、途中からtrainとvalidationの性能の差が大きくなり、過学習してしまっていることも確認できた。

In [ ]:
# [1-12]
# 学習曲線を描画する関数の作成
def plot_result(log):
    fig = plt.figure(figsize=(10, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(log.history["accuracy"])
    ax1.plot(log.history["val_accuracy"])
    ax1.set_title("Model accuracy")
    ax1.set_ylabel("Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.legend(["Train", "Validation"], loc="best")
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(log.history["loss"])
    ax2.plot(log.history["val_loss"])
    ax2.set_title("Model loss")
    ax2.set_ylabel("Loss")
    ax2.set_xlabel("Epoch")
    ax2.legend(["Train", "Validation"], loc="best")
    plt.show()

In [ ]:
# [1-13]
# 実際に学習曲線の確認
# 右が正解率を左が誤差の推移を表している。
plot_result(trainlog)

In [ ]:
# [1-14]
# 評価
loss_train, accuracy_train = model.evaluate(X_test, y_test, verbose=0)

# 学習前のスコアの表示
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

# 学習後のスコアの表示
print("===== 学習後 =====")
print(f"学習後の誤差 :  ", loss_train)
print(f"学習後の正解率 : ", accuracy_train)

---
## 学習する部分をより多くした　Fine-tuning

事前学習した重みを初期値として用い、事前学習した部分も学習することをFine-tuningと呼ぶ。
Fine-tuningは、全部の重みを更新することも含めるが、それに限らない。

今回は、Conv5の畳み込み層(block5_conv1 ～ block5_conv3)も含めて再学習（fine-tuning)を行い、精度を検証する。



<img src="https://drive.google.com/uc?export=view&id=11UF8Y7XxJYoAB2U53iishzZHGGiBeLbV" width = 50%></img>




先ほどのTransfer learningのモデルは85-87%程度で性能の限界が来た。
途中までは先ほどと同様。新しくモデルを定義する。(model 2)


In [ ]:
# [2-0]
# 先ほどと同様のnetwork をmodel2として再定義
model2 = get_vgg16_model(DATASET)

## モデルのセットアップ
FineTuningの場合は事前学習で得られた初期値を変更しすぎないように小さい学習率を用いる。

In [ ]:
# [2-1]
# 深いCNNの層とdenseの部分だけ学習するときはtrainingに"deep"を指定する。
model_setup(model2, training="deep", lr=0.00005)
# 学習する部分のパラメータ確認
model2.summary()

## モデルの学習
先ほどと同様に3 EPOCH学習する。
※こちらも上記と同様に授業外で10EPOCHの学習を行う。

In [ ]:
# [2-2]
EPOCH = 3
# ここを10にして再度実行する。
# EPOCH = 10
batch_size = 128

trainlog2 = model2.fit(
    X_train, y_train, epochs=EPOCH, validation_split=0.1, batch_size=batch_size
)

（参考）以下が10epoch学習を行った時のログ。レポートでは必ず自分の実施データに基づいた結果の報告及び考察を行うこと。
```
Epoch 1/10
352/352 [==============================] - 217s 615ms/step - loss: 1.0685 - accuracy: 0.6766 - val_loss: 0.4877 - val_accuracy: 0.8380
Epoch 2/10
352/352 [==============================] - 216s 613ms/step - loss: 0.3255 - accuracy: 0.8915 - val_loss: 0.3742 - val_accuracy: 0.8744
Epoch 3/10
352/352 [==============================] - 216s 613ms/step - loss: 0.1394 - accuracy: 0.9559 - val_loss: 0.3495 - val_accuracy: 0.8886
Epoch 4/10
352/352 [==============================] - 216s 614ms/step - loss: 0.0524 - accuracy: 0.9861 - val_loss: 0.3654 - val_accuracy: 0.8944
Epoch 5/10
352/352 [==============================] - 216s 614ms/step - loss: 0.0166 - accuracy: 0.9974 - val_loss: 0.3722 - val_accuracy: 0.9018
Epoch 6/10
352/352 [==============================] - 216s 614ms/step - loss: 0.0053 - accuracy: 0.9998 - val_loss: 0.3918 - val_accuracy: 0.9036
Epoch 7/10
352/352 [==============================] - 216s 614ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.4146 - val_accuracy: 0.9066
Epoch 8/10
352/352 [==============================] - 216s 614ms/step - loss: 9.1993e-04 - accuracy: 1.0000 - val_loss: 0.4433 - val_accuracy: 0.9078
Epoch 9/10
352/352 [==============================] - 216s 614ms/step - loss: 5.9912e-04 - accuracy: 1.0000 - val_loss: 0.4580 - val_accuracy: 0.9078
Epoch 10/10
352/352 [==============================] - 216s 614ms/step - loss: 4.2908e-04 - accuracy: 1.0000 - val_loss: 0.4702 - val_accuracy: 0.9090
```
畳み込み層を全て固定している転移学習よりもvalidationデータに対して高い性能が確認できた。

In [ ]:
# [2-3]
# 実際に学習曲線の確認
# 右が正解率を左が誤差の推移を表している。
plot_result(trainlog2)

In [ ]:
# [2-4]
# 評価
loss_train_ft, accuracy_train_ft = model2.evaluate(X_test, y_test, verbose=0)

# 学習前のスコアの表示
print("転移学習の誤差 : ", loss_train)
print("転移学習の正解率 : ", accuracy_train)

# 学習後のスコアの表示
print("===== 学習後 =====")
print(f'Fine Tuning後の誤差 :  ', loss_train_ft)
print(f'Fine Tuning後の正解率 : ', accuracy_train_ft)

独自の追加実験などは、以下のセルで行え。（いくつつかってもいい）
ただし、一度学習したモデルを再度学習すると、追加の学習になってしまうので注意。


In [ ]:
# ここから実験を追加する。(自由にセルを増やしてOK)

## 課題B2-5 転移学習やFineTuningと、それを行わなかった場合との比較

事前学習モデルを使わなかったとき（B2-4)と、今回の実験で転移学習やFine tuning、また独自に追加した異なる条件で行った実験により得られた結果を比較し、学習されるパラメータ数、学習時間、精度について考察せよ。

**（転移学習やFine-tuningを行った場合と、そうでない場合にどんな違いがあるかの方が重要。その上で、再学習する層の数の違いに着目する余裕があれば行う。）**

転移学習やFine-tuningを行うと、どんないいことがあるのか。またそれはいつでも行えるのか。行うための条件などがあるのか議論せよ。
